In [1]:
import requests
import json
from PIL import Image
import base64
import os
http_url = 'https://api-us.faceplusplus.com/facepp/v3/detect'
key = "7AUVsIF0R8T2YYZmrfe5-4r9FdumOFYc"
secret = "0TF9Xcz_9RVi10q_2Y6SqKdkiM8Fmlze"
image_path = "Rump.jpg"
image_name= os.path.splitext(image_path)[0]
with open(image_path, "rb") as img_file:
    img_string = base64.b64encode(img_file.read())
response = requests.post(
    'https://api-us.faceplusplus.com/facepp/v3/detect',
    {
        'api_key': key,
        'api_secret': secret,
        'image_base64': img_string,
        'return_landmark': 1,
        'return_attributes': 
'headpose,eyestatus,facequality,mouthstatus,eyegaze,blur,emotion'
    }
)
data = response.json()
print(data)
json_file=image_name + ".json"
with open(image_name + ".json", 'w') as outfile:
    json.dump(data, outfile)


{'request_id': '1606949545,fd4290c0-bad7-4101-99a7-76847ff59bd4', 'time_used': 117, 'faces': [{'face_token': '129f26dee3881694e36d114a38f7685c', 'face_rectangle': {'top': 156, 'left': 311, 'width': 202, 'height': 202}, 'landmark': {'contour_chin': {'x': 453, 'y': 352}, 'contour_left1': {'x': 310, 'y': 213}, 'contour_left2': {'x': 314, 'y': 237}, 'contour_left3': {'x': 321, 'y': 260}, 'contour_left4': {'x': 330, 'y': 282}, 'contour_left5': {'x': 343, 'y': 303}, 'contour_left6': {'x': 359, 'y': 322}, 'contour_left7': {'x': 379, 'y': 336}, 'contour_left8': {'x': 401, 'y': 347}, 'contour_left9': {'x': 426, 'y': 354}, 'contour_right1': {'x': 490, 'y': 177}, 'contour_right2': {'x': 494, 'y': 197}, 'contour_right3': {'x': 497, 'y': 218}, 'contour_right4': {'x': 497, 'y': 239}, 'contour_right5': {'x': 495, 'y': 260}, 'contour_right6': {'x': 492, 'y': 281}, 'contour_right7': {'x': 487, 'y': 301}, 'contour_right8': {'x': 481, 'y': 321}, 'contour_right9': {'x': 471, 'y': 339}, 'left_eye_bottom': 

In [2]:
import json
import numpy as np
import cv2
import os

### p1 left point, p2 right point
def point_on_line (p1, p2, ratio):
    x = p1[0] + ratio*(p2[0]-p1[0])
    y = p1[1] + ratio*(p2[1]-p1[1])
    return (int(x), int(y))

def get_ratio (p1, p2, p3):
    return (p3[0]-p1[0])/(p2[0]-p1[0])

def closest_point_on_line(p1, p2, p3):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    dx, dy = x2-x1, y2-y1
    det = dx*dx + dy*dy
    a = (dy*(y3-y1)+dx*(x3-x1))/det
    return (int(x1+a*dx), int(y1+a*dy))

### Create map of offset from reference point
def offset_map (landmarks, ref_pt):
    off_map = {}
    for feature in landmarks.keys():
        pt= (landmarks[feature]['x'], landmarks[feature]['y'])
        off_map[feature] = (pt[0] - ref_pt[0], pt[1] - ref_pt[1])
    return off_map

### Select Image
image_path = "pic2.jpg"
image_name= os.path.splitext(image_path)[0]
json_file=image_name + ".json"
with open(image_name + ".json") as f:
    data = json.load(f)
faces = data['faces']
img = cv2.imread(image_path)

### Mark all points on face
for face in faces:
    landmarks = face['landmark']
    for feature in landmarks.keys():
        x=landmarks[feature]['x']
        y=landmarks[feature]['y']
        img = cv2.circle(img, (x,y), radius=2, color=(0, 0, 255), thickness=-1)
        
### Line on bottom nose           
# for face in faces:
#     landmarks = face['landmark']
#     start = 'nose_tip'
#     end = 'nose_contour_lower_middle'
#     start_point = (landmarks[start]['x'], landmarks[start]['y'])
#     end_point = (landmarks[end]['x'], landmarks[end]['y'])
#     img = cv2.line(img, start_point, end_point, color=(0, 255, 0), thickness=1) 

### Marking contour lines
for face in faces:
    landmarks = face['landmark']
    for i in range(1, 10):
        start = 'contour_left' + str(i)
        end = 'contour_right' + str(i)
        ref = 'left_eye_pupil'
        start_point = (landmarks[start]['x'], landmarks[start]['y'])
        end_point = (landmarks[end]['x'], landmarks[end]['y'])
        img = cv2.line(img, start_point, end_point, color=(0, 255, 0), thickness=1) 

        
# ### May need to adjust offset to be proprotional with changes in contour lines
# ### Shift Left Eye
# for face in faces:
#     landmarks = face['landmark']    
#     start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
#     end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
#     ref = 'left_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
#     new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
#     new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
#     new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
#     new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
#     img = cv2.circle(img, new_ref, radius=2, color=(255, 0, 0), thickness=-1)
    
#     offsets = offset_map (landmarks, ref_point)
#     for feature in landmarks.keys():
#         if "left_eye" in feature:
#             offset = offsets[feature]
#             new_pt = (new_ref[0] + offset[0], new_ref[1] + offset[1])
#             img = cv2.circle(img, new_pt, radius=2, color=(255, 0, 0), thickness=-1)
            
# ### Shift Right Eye
# for face in faces:
#     landmarks = face['landmark']    
#     start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
#     end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
#     ref = 'right_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
#     new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
#     new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
#     new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
#     new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
#     img = cv2.circle(img, new_ref, radius=2, color=(255, 0, 0), thickness=-1)
    
#     offsets = offset_map (landmarks, ref_point)
#     for feature in landmarks.keys():
#         if "right_eye" in feature:
#             offset = offsets[feature]
#             new_pt = (new_ref[0] + offset[0], new_ref[1] + offset[1])
#             img = cv2.circle(img, new_pt, radius=2, color=(255, 0, 0), thickness=-1)
            
# ### Shift mouth            
# for face in faces:
#     landmarks = face['landmark']    
#     start = 'contour_left5'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
#     end = 'contour_right5'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
#     ref = 'mouth_upper_lip_top'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
#     new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
#     new_start = 'contour_left4'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
#     new_end = 'contour_right4'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
#     new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
#     img = cv2.circle(img, new_ref, radius=2, color=(255, 0, 0), thickness=-1)
    
#     offsets = offset_map (landmarks, ref_point)
#     for feature in landmarks.keys():
#         if "mouth" in feature:
#             offset = offsets[feature]
#             new_pt = (new_ref[0] + offset[0], new_ref[1] + offset[1])
#             img = cv2.circle(img, new_pt, radius=2, color=(255, 0, 0), thickness=-1)

cv2.imwrite(image_name + "_mapping.jpg", img) 
cv2.imshow('image',img)
cv2.waitKey(0)

-1

In [6]:
import json
import numpy as np
import cv2
import os

class HumanFace:
    landmarks = {}
    def __init__(self, face):
        self.landmarks = face['landmark']  

    def findCorners(self, points_dict):
        left, top = 2147483646, 2147483646
        right, bottom = 0, 0
        for feat, coord in points_dict.items():
            if coord['x'] < left: 
                left = coord['x']
            if coord['x'] > right: 
                right = coord['x']
            if coord['y'] < top: 
                top = coord['y']
            if coord['y'] > bottom: 
                bottom = coord['y']
        return left, right, bottom, top

    def get_dict(self, keyword):
        key_list = []
        for feature in self.landmarks.keys():
            if keyword in feature:
                key_list.append(feature)
        feature_dict = {k:v for k,v in self.landmarks.items() if k in key_list}
        return feature_dict

    def get_object_region(self, body_part):
        body_dict = self.get_dict(body_part)
        left, right, bottom, top = self.findCorners(body_dict)
        region_dict = {'left': left, 'right': right, 'top': top, 'bottom': bottom}
        return region_dict
    
#     def get_points(self):
        
### Create map of offset from reference point
def get_offset (points, ref_pt):
    offsets = []
    for pt in points:
        offsets.append((pt[0] - ref_pt[0], pt[1] - ref_pt[1]))
    return offsets

def apply_inpaint(img, top, bottom, left, right):
    mask = img.copy()
    mask[:,:] = 0
    mask[top:bottom,left:right] = 255
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    dst = cv2.inpaint(img,mask,25,cv2.INPAINT_TELEA)
    
    blurred_mask = img.copy()
    blurred_mask[:,:] = 0
    blurred_mask[top:bottom,left:right]=255
    blurred_mask = cv2.GaussianBlur(blurred_mask,(55,55),0)
    blurred_mask = blurred_mask.astype('float') / 255.
    img = img.astype('float') / 255.
    dst = dst.astype('float') / 255.
    img = img * (1 - blurred_mask) + dst * blurred_mask
    img = (img * 255).astype('uint8')
    return img

def ray_tracing(y,x,poly):
    n = len(poly)
    inside = False
    p2x = 0.0
    p2y = 0.0
    xints = 0.0
    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside
    
### Select Image
image_path = "pic3.jpg"
image_name= os.path.splitext(image_path)[0]
json_file=image_name + ".json"
with open(image_name + ".json") as f:
    data = json.load(f)
faces = data['faces']
img = cv2.imread(image_path)

### Remove Nose
for face in faces:
    subject = HumanFace(face) 
    nose_region = subject.get_object_region('nose')
    point1 = (nose_region['left'] - 5, nose_region['top'] - 5)
    point2 = (nose_region['right'] + 5, nose_region['bottom'] + 5)
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    ### Apply Inpaint
    img = apply_inpaint(img, point1[1], point2[1], point1[0], point2[0])
    

### Shift Left Eye
for face in faces:
    subject = HumanFace(face) 
    left_eye_region = subject.get_object_region('left_eye')
    point1 = (left_eye_region['left'] - 5, left_eye_region['top'] - 5)
    point2 = (left_eye_region['right'] + 5, left_eye_region['bottom'] + 10)
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'left_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    
    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    left_eye = img[point1[1]:point2[1], point1[0]:point2[0]]
    left_eye = left_eye.copy()
    
    ### Apply Inpaint
    img = apply_inpaint(img, point1[1], point2[1], point1[0], point2[0])
    
    eye_order = ['left_eye_bottom', 'left_eye_left_corner', 
                 'left_eyebrow_left_corner', 'left_eyebrow_upper_left_quarter',
                 'left_eyebrow_upper_middle', 'left_eyebrow_upper_right_quarter',
                 'left_eyebrow_right_corner',
                 'left_eye_right_corner'
                ]

    eye_dict = subject.get_dict('left_eye')
    offsets = offset_map (eye_dict, ref_point)
    ### Index 0: Horizontal | Index 1: Vertical
    eye_bound = {'left_eye_bottom': (0, 5),
                 'left_eye_left_corner': (-5, 3), 
                 'left_eyebrow_left_corner': (-5, 0),
                 'left_eyebrow_upper_left_quarter': (0, -5),
                 'left_eyebrow_upper_middle': (0, -5),
                 'left_eyebrow_upper_right_quarter': (0, -5),
                 'left_eyebrow_right_corner': (5, 0),
                 'left_eye_right_corner': (5, 3)
                  }

    for feature in eye_bound.keys():
        offset = offsets[feature]
        bound_shift = eye_bound[feature]
        eye_bound[feature] = (new_ref[0] + offset[0] + bound_shift[0] - new_point1[0], new_ref[1] + offset[1] + bound_shift[1] - new_point1[1])

    polygon = []
    for feature in eye_order:
        polygon.append(eye_bound[feature])

    temp_img = img.copy()
    img_block = temp_img[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]]
    for i in range(img_block.shape[0]):
        for j in range(img_block.shape[1]):
#             if ray_tracing(i, j, polygon):
            img_block[i][j] = left_eye[i][j]
    
    blurred_mask = img.copy()
    blurred_mask[:,:] = 0
    blurred_mask[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]] = 255            
    blurred_mask = cv2.GaussianBlur(blurred_mask,(99,99),0)
    blurred_mask = blurred_mask.astype('float') / 255.
    temp_img = temp_img.astype('float') / 255.
    img = img.astype('float') / 255.
    img = img * (1 - blurred_mask) + temp_img * blurred_mask
    img = (img * 255).astype('uint8')


### Shift Right Eye
for face in faces:
    subject = HumanFace(face) 
    right_eye_region = subject.get_object_region('right_eye')
    point1 = (right_eye_region['left'] - 5, right_eye_region['top'] - 5)
    point2 = (right_eye_region['right'] + 5, right_eye_region['bottom'] + 10)
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]

    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'right_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)

    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))

    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    right_eye = img[point1[1]:point2[1], point1[0]:point2[0]]
    right_eye = right_eye.copy()

    ### Apply Inpaint
    img = apply_inpaint(img, point1[1], point2[1], point1[0], point2[0])
    
    eye_order = ['right_eye_bottom', 'right_eye_left_corner', 
                 'right_eyebrow_left_corner', 'right_eyebrow_upper_left_quarter',
                 'right_eyebrow_upper_middle', 'right_eyebrow_upper_right_quarter',
                 'right_eyebrow_right_corner',
                 'right_eye_right_corner'
                ]
    eye_dict = subject.get_dict('right_eye')
    offsets = offset_map (eye_dict, ref_point)
    ### Index 0: Horizontal | Index 1: Vertical
    eye_bound = {'right_eye_bottom': (0, 5),
                 'right_eye_left_corner': (-5, 3), 
                 'right_eyebrow_left_corner': (-5, 0),
                 'right_eyebrow_upper_left_quarter': (0, -5),
                 'right_eyebrow_upper_middle': (0, -5),
                 'right_eyebrow_upper_right_quarter': (0, -5),
                 'right_eyebrow_right_corner': (5, 0),
                 'right_eye_right_corner': (5, 3)
                  }

    for feature in eye_bound.keys():
        offset = offsets[feature]
        bound_shift = eye_bound[feature]
        eye_bound[feature] = (new_ref[0] + offset[0] + bound_shift[0] - new_point1[0], new_ref[1] + offset[1] + bound_shift[1] - new_point1[1])

    polygon = []
    for feature in eye_order:
        polygon.append(eye_bound[feature])
        
    temp_img = img.copy()
    img_block = temp_img[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]]
    for i in range(img_block.shape[0]):
        for j in range(img_block.shape[1]):
#             if ray_tracing(i, j, polygon):
                img_block[i][j] = right_eye[i][j]
    
    blurred_mask = img.copy()
    blurred_mask[:,:] = 0
    blurred_mask[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]] = 255            
    blurred_mask = cv2.GaussianBlur(blurred_mask,(99,99),0)
    blurred_mask = blurred_mask.astype('float') / 255.
    temp_img = temp_img.astype('float') / 255.
    img = img.astype('float') / 255.
    img = img * (1 - blurred_mask) + temp_img * blurred_mask
    img = (img * 255).astype('uint8')


### Shift Mouth
for face in faces:
    subject = HumanFace(face) 
    mouth_region = subject.get_object_region('mouth')
    point1 = (mouth_region['left'] - 10, mouth_region['top'] - 8)
    point2 = (mouth_region['right'] + 10, mouth_region['bottom'] + 12)
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]

    start = 'contour_left5'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right5'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'mouth_lower_lip_top'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)

    new_start = 'contour_left4'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right4'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))

    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    mouth = img[point1[1]:point2[1], point1[0]:point2[0]]
    mouth = mouth.copy()

    ### Apply Inpaint
    img = apply_inpaint(img, point1[1], point2[1], point1[0], point2[0])


    mouth_order = ['mouth_left_corner', 'mouth_upper_lip_left_contour2', 'mouth_upper_lip_left_contour1',
                   'mouth_upper_lip_top',
                   'mouth_upper_lip_right_contour1', 'mouth_upper_lip_right_contour2', 'mouth_right_corner', 
                   'mouth_lower_lip_right_contour2', 'mouth_lower_lip_right_contour3',
                   'mouth_lower_lip_bottom',
                  'mouth_lower_lip_left_contour3', 'mouth_lower_lip_left_contour2']
    mouth_dict = subject.get_dict('mouth')
    offsets = offset_map (mouth_dict, ref_point)
    ### Index 0: Horizontal | Index 1: Vertical
    mouth_bound = {'mouth_left_corner': (-5, 0), 
                   'mouth_upper_lip_left_contour2': (0, -5),
                   'mouth_upper_lip_left_contour1': (0, -3),
                   'mouth_upper_lip_top': (0, -3),
                   'mouth_upper_lip_right_contour1': (0, -3), 
                   'mouth_upper_lip_right_contour2': (0, -5),
                   'mouth_right_corner': (5, 0),
                   'mouth_lower_lip_right_contour2': (0, 5), 
                   'mouth_lower_lip_right_contour3': (0, 5),
                   'mouth_lower_lip_bottom': (0, 5),
                   'mouth_lower_lip_left_contour3': (0, 5),
                   'mouth_lower_lip_left_contour2': (0, 5)
                  }

    for feature in mouth_bound.keys():
        offset = offsets[feature]
        bound_shift = mouth_bound[feature]
        mouth_bound[feature] = (new_ref[0] + offset[0] + bound_shift[0] - new_point1[0], new_ref[1] + offset[1] + bound_shift[1] - new_point1[1])

    polygon = []
    for feature in mouth_order:
        polygon.append(mouth_bound[feature])
        
    temp_img = img.copy()
    img_block = temp_img[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]]
#     img_block = img[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]]

    mask = 255 * np.ones(mouth.shape, mouth.dtype)
    center = (int((new_point1[0]+new_point2[0])/2), int((new_point1[1]+new_point2[1])/2))
    for i in range(img_block.shape[0]):
        for j in range(img_block.shape[1]):
            if not ray_tracing(i, j, polygon):
                mask[i][j] = mask[i][j] * 0
    img = cv2.seamlessClone(mouth, img, mask, center, cv2.NORMAL_CLONE)
        
#     blurred_mask = img.copy()
#     blurred_mask[:,:] = 0
#     blurred_mask[new_point1[1]:new_point2[1], new_point1[0]:new_point2[0]] = 255            
#     blurred_mask = cv2.GaussianBlur(blurred_mask,(99,99),0)
#     blurred_mask = blurred_mask.astype('float') / 255.
#     temp_img = temp_img.astype('float') / 255.
#     img = img.astype('float') / 255.
#     img = img * (1 - blurred_mask) + temp_img * blurred_mask
#     img = (img * 255).astype('uint8')
    
    ## Visual
#     for feature in mouth_bound.keys():
#         new_pt = mouth_bound[feature]
#         img = cv2.circle(img, new_pt, radius=2, color=(255, 0, 0), thickness=-1)

cv2.imwrite(image_name + "_output.jpg", img) 
cv2.imshow('image',img)
cv2.waitKey(0)
            

-1

In [84]:
def ray_tracing(y,x,poly):
    n = len(poly)
    inside = False
    p2x = 0.0
    p2y = 0.0
    xints = 0.0
    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y

    return inside

x = (0, 1)
y = (2, 3)
z = (x[0] + y[0], x[1] + y [1])
print(z)
# ### Select Image
# image_path = "pic3.jpg"
# image_name= os.path.splitext(image_path)[0]
# json_file=image_name + ".json"
# with open(image_name + ".json") as f:
#     data = json.load(f)
# faces = data['faces']
# img = cv2.imread(image_path)

# polygon = [(200, 300), (400, 300), (300, 500)]
# mask = np.full((img.shape[0], img.shape[1]), 0, dtype=np.uint8)
# mask[:,:] = 0
# for i in range(img.shape[0]):
#     for j in range(img.shape[1]):
#         if ray_tracing(i, j, polygon) :
#             mask[i][j] = 1 
            
# img = cv2.bitwise_and(img, mask)            
# # for i in range(img.shape[0]):
# #     for j in range(img.shape[1]):
# #         if mask[i][j] == 255 :
# #             img[i][j] = (255, 255, 255)
# cv2.imshow('image',img)
# cv2.waitKey(0)

(2, 4)
